We first import the necessary packages

In [ ]:
# install libraries
!pip install scikeras
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from tensorflow import keras
import tensorflow as tf
import random as python_random
from sklearn.pipeline import Pipeline
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from scikeras.wrappers import KerasClassifier


We load the train and test data sets.

In [ ]:
# load dataset and transform to pandas dataframe
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


We one hot encode our data, the number of classes are 10 (0 through 9).

In [ ]:
y_train_encoded = to_categorical(y_train, num_classes=10)
y_test_encoded = to_categorical(y_test, num_classes=10)

Function to create a sequntial model to be trained.

In [ ]:
def create_model(learning_rate=0.001, num_filters=16):
  model = keras.models.Sequential()
  #Define filters and convolutional layers here
  model.add(keras.layers.Conv2D(filters=num_filters, kernel_size=(3, 3),
  activation='relu', input_shape=(28, 28, 1)))
  #Add a maxpooling layer
  model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
  #Flatten the output and give it to a fully connected layer
  model.add(keras.layers.Flatten())
  #One hidden layer maps the flattened neurons to output
  model.add(keras.layers.Dense(10, activation='softmax'))
  optimizer = keras.optimizers.Adam(learning_rate=learning_rate)  # Set the learning rate
  model.compile(optimizer='adam', loss='categorical_crossentropy',
  metrics=['accuracy'])
  return model


We iterate through 4 combinations of learning rates and number of filters.
We use stratified k fold with k = 5 during parameter exploration for each set of paramters.
Best paramters are determined using highest f1 score of the 4 combinations.
We use our training data for Stratified K fold cross validation.


In [ ]:
# initialize stratified k fold

learning_rates = [0.001, 0.01]
num_filters = [16, 32]

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = []

for lr in learning_rates:
    for nf in num_filters:
        # Create a Keras model for the current hyperparameters
        model = create_model(learning_rate=lr, num_filters=nf)

        # Create a Scikit-learn pipeline
        pipeline = sk.pipeline.Pipeline([
            ('classifier', KerasClassifier(model=create_model, epochs=10, batch_size=32, verbose=0))
        ])

        # Lists to store results for each fold
        f1_scores = []

        for train_index, test_index in skf.split(X_train, y_train):
            X_train_fold, X_test_fold = X_train[train_index], X_train[test_index]
            y_train_fold, y_test_fold = y_train_encoded[train_index], y_train_encoded[test_index]

            # Fit the pipeline on the training data
            pipeline.fit(X_train_fold, y_train_fold)

            # Predict on the test data
            y_pred = pipeline.predict(X_test_fold)

            # Calculate the F1 score for this fold
            f1 = f1_score(y_true = y_test_fold.argmax(axis=1), y_pred = y_pred.argmax(axis=1), average='macro')
            f1_scores.append(f1)

        # Calculate the mean F1 score for these hyperparameters
        mean_f1 = np.mean(f1_scores)
        results.append((lr, nf, mean_f1))
        print(f"Learning Rate: {lr}, Num Filters: {nf}, Mean F1 Score: {mean_f1}")

# Find the hyperparameters with the best F1 score
best_params = max(results, key=lambda x: x[2])
print(f"Best Hyperparameters: Learning Rate {best_params[0]}, Num Filters {best_params[1]}, Mean F1 Score {best_params[2]}")




We now train a new model on the best parameters and the entire data set inside a pipeline.

We then test it on the test MNIST data set loaded earlier.

We report the classification accuracy as this is one of the most commonly used metrics for MNIST data sets.

In [ ]:
# Best hyperparameters
best_lr, best_nf, _ = best_params

# Create a Keras model with the best hyperparameters
best_model = create_model(learning_rate=best_lr, num_filters=best_nf)

# Best hyperparameters
best_lr, best_nf, _ = best_params

# Create a Keras model with the best hyperparameters
best_model = create_model(learning_rate=best_lr, num_filters=best_nf)

# Create a Scikit-learn pipeline with the best model
pipeline = sk.pipeline.Pipeline([
    ('classifier', KerasClassifier(model=best_model, epochs=10, batch_size=32, verbose=0))
])


# Fit the pipeline with the best model on the full training dataset
pipeline.fit(X_train, y_train_encoded)

# Evaluate the model on the test dataset using the pipeline and get accuracy
test_accuracy = pipeline.score(X_test, y_test_encoded)

# Report the classification accuracy on the test dataset
print(f"Classification Accuracy for learning rate {best_lr} and number of neurons {best_nf}: {test_accuracy}")
